### Import required libraries

In [ ]:
import os
import re
import xml.etree.ElementTree as et
import pandas as pd

### Set up download directory and extract the data from all XML files, to then place it in a dataframe

In [ ]:
# Directory containing XML files
download_directory = r"Your_directory"

# List to store the extracted data
data = []

# Updated regular expression to extract reference number from the filename
reference_number_pattern = re.compile(r'_(HR[AB])_(\d+)(_[a-zA-Z0-9]+)?\+')

# Iterate over XML files in the download directory
for xml_file in os.listdir(download_directory):
    if xml_file.endswith(".xml"):
        xml_file_path = os.path.join(download_directory, xml_file)
        
        with open(xml_file_path, "r", encoding="utf-8") as file:
            xml_content = file.read()

        try:
            # Parse the XML content
            xml_tree = et.fromstring(xml_content)

            # Define the namespaces
            namespaces = {
                'xjustiz': 'http://www.xjustiz.de'
            }

            # Extract the company name
            company_name_elem = xml_tree.find(".//xjustiz:grunddaten/xjustiz:verfahrensdaten/xjustiz:beteiligung/xjustiz:beteiligter/xjustiz:auswahl_beteiligter/xjustiz:organisation/xjustiz:bezeichnung/xjustiz:bezeichnung.aktuell", namespaces)
            company_name = company_name_elem.text if company_name_elem is not None else "Unknown"

            if company_name == "Unknown":
                print(f"Company name not found in file: {xml_file}")

            # Extract the reference number from the filename
            match = reference_number_pattern.search(xml_file)
            if match:
                reference_number = f"{match.group(1)}_{match.group(2)}"
            else:
                reference_number = "Unknown"
               
            # Extract the address details
            house_number_elem = xml_tree.find(".//xjustiz:grunddaten/xjustiz:verfahrensdaten/xjustiz:beteiligung/xjustiz:beteiligter/xjustiz:auswahl_beteiligter/xjustiz:organisation/xjustiz:anschrift/xjustiz:hausnummer", namespaces)
            street_elem = xml_tree.find(".//xjustiz:grunddaten/xjustiz:verfahrensdaten/xjustiz:beteiligung/xjustiz:beteiligter/xjustiz:auswahl_beteiligter/xjustiz:organisation/xjustiz:anschrift/xjustiz:strasse", namespaces)
            postal_code_elem = xml_tree.find(".//xjustiz:grunddaten/xjustiz:verfahrensdaten/xjustiz:beteiligung/xjustiz:beteiligter/xjustiz:auswahl_beteiligter/xjustiz:organisation/xjustiz:anschrift/xjustiz:postleitzahl", namespaces)
            place_elem = xml_tree.find(".//xjustiz:grunddaten/xjustiz:verfahrensdaten/xjustiz:beteiligung/xjustiz:beteiligter/xjustiz:auswahl_beteiligter/xjustiz:organisation/xjustiz:anschrift/xjustiz:ort", namespaces)
            
            house_number = house_number_elem.text if house_number_elem is not None else "Unknown"
            street = street_elem.text if street_elem is not None else "Unknown"
            postal_code = postal_code_elem.text if postal_code_elem is not None else "Unknown"
            place = place_elem.text if place_elem is not None else "Unknown"

            # Extract capital values
            stammkapital_value = xml_tree.find(".//xjustiz:fachdatenRegister/xjustiz:auswahl_zusatzangaben/xjustiz:kapitalgesellschaft/xjustiz:zusatzGmbH/xjustiz:stammkapital/xjustiz:zahl", namespaces)
            grundkapital_value = xml_tree.find(".//xjustiz:fachdatenRegister/xjustiz:auswahl_zusatzangaben/xjustiz:kapitalgesellschaft/xjustiz:zusatzAktiengesellschaft/xjustiz:grundkapital/xjustiz:hoehe/xjustiz:zahl", namespaces)
            hafteinlage_value = xml_tree.find(".//xjustiz:fachdatenRegister/xjustiz:auswahl_zusatzangaben/xjustiz:personengesellschaft/xjustiz:zusatzGmbH/xjustiz:datenKommanditist/xjustiz:hafteinlage/xjustiz:zahl", namespaces)
            if hafteinlage_value is None:
                hafteinlage_value = xml_tree.find(".//xjustiz:fachdatenRegister/xjustiz:auswahl_zusatzangaben/xjustiz:personengesellschaft/xjustiz:zusatzKG/xjustiz:datenKommanditist/xjustiz:hafteinlage/xjustiz:zahl", namespaces)
            
            stammkapital = stammkapital_value.text if stammkapital_value is not None else "0"
            grundkapital = grundkapital_value.text if grundkapital_value is not None else "0"
            hafteinlage = hafteinlage_value.text if hafteinlage_value is not None else "0"
        
            # Add the extracted data to the list
            data.append({
                "Name": company_name,
                "Reference_number": reference_number,
                "Stammkapital": stammkapital,
                "Hafteinlage": hafteinlage,
                "Grundkapital": grundkapital,
                "House_number": house_number,
                "Street": street,
                "Postal_code": postal_code,
                "Place": place
            })

        except et.ParseError as e:
            print(f"Error parsing XML file {xml_file}: {e}")
            continue

# Create a DataFrame from the collected data
df = pd.DataFrame(data)

# Display the final DataFrame
print(df)